<a href="https://colab.research.google.com/github/Gurdeep-kaur533/Movie-Recommender-System/blob/main/movie_recommander_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")

In [ ]:
movies.head(1)

In [ ]:
credits.head(1)

In [ ]:
movies = movies.merge(credits,on='title')
movies.head(1)

In [ ]:
# geners, id, keywords, title, overview, cast, crew
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

In [ ]:
movies.info()

In [ ]:
movies.head()

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace = True)

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.iloc[0].genres

In [ ]:
# '[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'
# ['Action' ,'Adventure','Fantasy', 'Science Fiction'

In [ ]:
import ast

In [ ]:
def convert(obj):
   L = []
   for i in ast.literal_eval(obj):
       L.append(i['name'])
   return L

In [ ]:
movies['genres'] = movies['genres'].apply(convert)

In [ ]:
movies['genres']

In [ ]:
movies.head()

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)

In [ ]:
movies.head()

In [ ]:
movies['cast'][0]       # we need actual names not characters

In [ ]:
def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:                            # 3 used to display 3 dictionary data
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

In [ ]:
movies['cast'] = movies['cast'].apply(convert3)

In [ ]:
movies['cast']

In [ ]:
movies.head()

In [ ]:
movies['crew'][0]        # we need those dictionaries only in which job = director

In [ ]:
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
           L.append(i['name'])
           break
    return L

In [ ]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [ ]:
movies['crew']

In [ ]:
movies.head()

In [ ]:
movies['overview'][0]       # it is a string

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [ ]:
movies['overview']

In [ ]:
# now the problem is space like: sam worthington is change to samworthington   so tags are distributed to specific tags

In [ ]:
movies['genres'] = movies['genres'].apply(lambda x: [i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x: [i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x: [i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x: [i.replace(" ","") for i in x])

In [ ]:
movies.head()

In [ ]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [ ]:
movies.head()

In [ ]:
new_df = movies[['movie_id', 'title', 'tags']]

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
new_df.head()

In [ ]:
new_df['tags'][0]

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())

In [ ]:
new_df.head()     # in lower case

In [ ]:
import nltk
!pip install nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()                               # return same word like dance to danc and loving to love

In [ ]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)

In [ ]:
new_df['tags'][0]

In [ ]:
new_df['tags'][1]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000,stop_words = 'english')

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()       # like sparse matrix

In [ ]:
vectors

In [ ]:
vectors[0]

In [ ]:
cv.get_feature_names_out()

In [ ]:
ps.stem('danced')

In [ ]:
stem('in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver jamescameron')

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors)              # distance(cosine) between movies(vectors)

In [ ]:
sorted(list(enumerate(similarity[0])),reverse = True,key = lambda x: x[1]) [1:11]      # index position created with distances

In [ ]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse = True,key = lambda x: x[1]) [1:11]

    for i in movies_list:
        print(new_df.iloc[i[0]].title)                       # fetch similar 10 movies title

In [ ]:
recommend('Avatar')

In [ ]:
recommend('Batman Begins')

In [ ]:
new_df.iloc[1214].title

In [ ]:
import pickle

In [ ]:
pickle.dump(new_df,open('movies.pkl','wb'))

In [ ]:
new_df['title'].values

In [ ]:
pickle.dump(similarity,open('similarity.pkl', 'wb'))